Here are some references to try to speed up the loading:
* [Fastest way to load data into PostgreSQL using Python](https://hakibenita.com/fast-load-data-python-postgresql).



In [1]:
import os
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from credentials import pmc_credentials # a dict defined in a python file in the current directory

connection_string = 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(**pmc_credentials)

engine = create_engine(connection_string)

# pd.read_sql("select * from paragraph order by pmid, paragraph_number limit 5", con=engine)

# Import model data

In [3]:
# https://github.com/pgvector/pgvector
# We have already enabled pgvector for this database by running `CREATE EXTENSION vector;` as postgres superuser.
# Create the target table as pmc_admin:
sql = """
CREATE TABLE model_data (
    pattern_name text,
    intercept real,
    vlen real,
    coef_unit_vector vector(768)
);
"""

In [5]:
model_data_file = "../bloggable/coefficients/data/PMC_logistic_regression_model_data.parquet"
model_data = pd.read_parquet(model_data_file)
model_data['intercept'] = [v[0] for v in model_data['intercept']]
model_data

,pattern_name,intercept,vlen,coef_unit_vector
0,TITLE,1.689223,666.064767,"[0.00783694987774217, 0.030580348537854546, 0...."
1,ABSTRACT,-2.670814,187.571289,"[-0.0020985786538986902, 0.08352741358268558, ..."
2,INTRODUCTION,-2.207903,102.715457,"[-0.001825770232056595, 0.0034384100827333495,..."
3,METHODS,-5.064921,83.507210,"[-0.008170266892882349, -0.05228882273387169, ..."
4,DISCUSSION,-2.855653,118.872954,"[-0.034243196269582485, -0.026812014959896827,..."
5,CONCLUSION,-3.701810,171.174707,"[0.035744067776705184, 0.07716460891329262, -0..."
6,ADVERSE_EVENTS,-8.690852,17.213403,"[0.023598664192919047, 0.001427428597294151, 0..."


In [7]:
model_data['coef_unit_vector'].values

array([array([ 7.83694988e-03,  3.05803485e-02,  3.55227019e-02,  5.78341437e-03,
               3.81735447e-02, -1.61349738e-02, -3.23622202e-02,  4.92367124e-03,
               5.13706352e-03,  2.91778328e-02, -3.35627640e-03, -1.01167431e-02,
               2.20061409e-03,  2.30614163e-02,  3.04039922e-02,  3.67442710e-02,
               5.74750760e-02,  5.38481541e-03,  8.93395582e-02,  2.50130670e-02,
              -4.27374985e-03,  2.00062302e-02,  3.52680580e-02,  4.73364175e-02,
               1.53593395e-02,  1.81955608e-02,  6.67983942e-03,  3.33251307e-02,
               9.76468508e-03,  9.16852511e-03,  3.24288527e-02,  2.15960839e-02,
               1.75271450e-02,  1.20265414e-02, -1.04006025e-04,  4.40157402e-02,
               4.28938310e-02,  4.46898154e-02,  8.42199846e-03,  2.92342046e-02,
               1.30837018e-02, -1.53539646e-02,  8.24046585e-03,  1.96689553e-02,
               3.58224209e-02,  1.49754023e-02,  2.54197129e-02,  1.67629375e-02,
               2

In [8]:
# model_data['coef_unit_vector'] # array of arrays
# can't adapt type 'numpy.ndarray'

# model_data['coef_unit_vector'] = [v for v in model_data['coef_unit_vector']]  # list of arrays
# ProgrammingError: can't adapt type 'numpy.ndarray'


model_data['coef_unit_vector'] = [v.tolist() for v in model_data['coef_unit_vector']]

In [9]:
model_data['coef_unit_vector'].values

array([list([0.00783694987774217, 0.030580348537854546, 0.03552270194927901, 0.005783414366768307, 0.038173544715908496, -0.01613497378603884, -0.03236222019262683, 0.0049236712351489725, 0.005137063517128661, 0.02917783280010229, -0.00335627639528537, -0.010116743127023237, 0.00220061408610614, 0.02306141629020915, 0.03040399222770465, 0.036744271003193, 0.057475075986561174, 0.005384815411832926, 0.0893395582202683, 0.02501306700786538, -0.004273749847358453, 0.020006230189783248, 0.035268057950224994, 0.04733641753248357, 0.015359339491254285, 0.018195560813845865, 0.0066798394182966325, 0.03332513068668363, 0.009764685080198343, 0.009168525109562902, 0.03242885274249551, 0.02159608392654329, 0.017527144958919747, 0.01202654136289859, -0.00010400602453817121, 0.0440157402166991, 0.0428938309784505, 0.044689815421505, 0.008421998461876213, 0.029234204639964574, 0.01308370176922106, -0.015353964595609467, 0.008240465854124177, 0.019668955278172546, 0.03582242088640441, 0.0149754023447

In [20]:
model_data.to_sql(
    name="model_data",
    con=engine,
    if_exists="append",
    index=False
)

7

In [21]:
pd.read_sql("select * from model_data", con=engine)

,pattern_name,intercept,vlen,coef_unit_vector
0,TITLE,1.689223,666.064760,"[0.00783695,0.03058035,0.035522703,0.005783414..."
1,ABSTRACT,-2.670814,187.571290,"[-0.0020985787,0.083527416,-0.034449317,-0.015..."
2,INTRODUCTION,-2.207903,102.715454,"[-0.0018257702,0.00343841,0.0069657504,0.03321..."
3,METHODS,-5.064920,83.507210,"[-0.008170267,-0.052288823,0.04312862,-0.02226..."
4,DISCUSSION,-2.855653,118.872955,"[-0.034243196,-0.026812015,-0.051408693,0.0159..."
5,CONCLUSION,-3.701809,171.174710,"[0.035744067,0.077164605,-0.045649808,0.054659..."
6,ADVERSE_EVENTS,-8.690852,17.213404,"[0.023598664,0.0014274286,0.05254408,0.0173189..."


In [23]:
sql = "SELECT * FROM model_data ORDER BY coef_unit_vector <-> (select coef_unit_vector from model_data where pattern_name='DISCUSSION') LIMIT 3;"

pd.read_sql(sql, con=engine)

,pattern_name,intercept,vlen,coef_unit_vector
0,DISCUSSION,-2.855653,118.872955,"[-0.034243196,-0.026812015,-0.051408693,0.0159..."
1,CONCLUSION,-3.701809,171.174710,"[0.035744067,0.077164605,-0.045649808,0.054659..."
2,INTRODUCTION,-2.207903,102.715454,"[-0.0018257702,0.00343841,0.0069657504,0.03321..."


In [ ]:
# %%time

# # This demonstrates that we can save list fields with pd.to_sql. (I have since moved this logic to pure sql.)
# repeated_pmids_csv = "repeated_pmids.csv"
# if os.file.exists(repeated_pmids_csv):
#     repeated_pmids = pd.read_csv(repeated_pmids_csv)
# else:
#     query = """
#     with p2 as (
#         select pmid,
#                 count(*) num_titles, 
#                 array_agg(text) title_list,
#                 array_agg(pmid) pmid_list, 
#                 array_agg(segment) segment_list 
#             from paragraph 
#                 where paragraph_number=0 group by pmid 
#                 order by num_titles desc 
#                 limit 75
#     )
#     select * from p2 where num_titles > 1
#     """
    
#     repeated_pmids = pd.read_sql(query, con=engine)
#     repeated_pmids.to_csv("repeated_pmids.csv", index=False)


# repeated_pmids.to_sql(name="repeated_pmids", con=engine)
# pd.read_sql("select * from repeated_pmids", con=engine)
# # 4min 50s

In [ ]:
# https://medium.com/@askintamanli/fastest-methods-to-bulk-insert-a-pandas-dataframe-into-postgresql-2aa2ab6d2b24
# https://stackoverflow.com/questions/23103962/how-to-write-dataframe-to-postgres-table

import pandas as pd
from sqlalchemy import create_engine
import time
import csv
from io import StringIO

def psql_insert_copy(table, conn, keys, data_iter): #mehod
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)


start_time = time.time() # get start time before insert

repeated_pmids.to_sql(
    name="repeated_pmids",
    con=engine,
    if_exists="append",
    index=False,
    method=psql_insert_copy
)

end_time = time.time() # get end time after insert
total_time = end_time - start_time # calculate the time
print(f"Insert time: {total_time} seconds") # print time

In [ ]:
model_data_recovered = pd.read_sql("select * from model_data", con=engine)
model_data_recovered

In [ ]:
type(model_data_recovered['coef_unit_vector'][0])

In [ ]:
type(model_data['coef_unit_vector'][0])